In [ ]:
import os
import base64
import pymupdf


from PIL import Image
from io import BytesIO
from openai import OpenAI
from dotenv import load_dotenv

load_dotenv()

True

In [ ]:
def encode_image(pdf_path):
    base64_list = []
    pdf_document = pymupdf.open(pdf_path)
    for page in pdf_document:
        pix = page.get_pixmap()
        img = Image.frombytes("RGB", [pix.width, pix.height], pix.samples)
        buffered = BytesIO()
        img.save(buffered, format="JPEG")
        omage_bytes = buffered.getvalue()
        base64_image = base64.b64encode(omage_bytes).decode('utf-8')
        base64_list.append(base64_image)
        
    return base64_list

In [ ]:
def extract_pdf(base64_images):

    client = OpenAI(
        api_key=os.getenv("GROQ_API_KEY"), 
        base_url="https://api.groq.com/openai/v1"
    )

    ocr_response = client.chat.completions.create(
        model="meta-llama/llama-4-maverick-17b-128e-instruct",
        messages=[
            {
                "role": "user", "content":[
                    {"type": "text", "text": "Extract the text from the image."},
                    {"type": "image_url", "image_url": {"url": f"data:image/jpeg;base64,{base64_images}"}}
                ]
            }
        ],
        max_tokens=2000
    )

    return ocr_response.choices[0].message.content


In [5]:
path = r"..\data\pdf\Understanding Credit Cards Fraud.pdf"

In [8]:
base64_list = encode_image(path)
base64_list

['...']

In [9]:
text = []
for base64string in base64_list:
    text.append(extract_pdf(base64string))

print(''.join(text))

The image displays a title page for a document titled "Understanding Credit Card Frauds." The title is prominently centered on the page, with the subtitle "Cards Business Review#2003-01" above it. The authors' names, "Tej Paul Bhatla, Vikram Prabhu & Amit Dua," are listed below the title, followed by the date "June 2003."

At the bottom of the page, a copyright notice reads, "© Tata Consultancy Services 2002. All rights reserved." The background of the image is white, providing a clean and simple visual presentation.

Overall, the image appears to be a professional document cover page, likely from a business or academic publication.The image shows a table of contents for a document about credit card fraud. The main topics and their corresponding page numbers are as follows:

*   **OVERVIEW**
    *   INTRODUCTION .......................................................1
    *   PURPOSE OF THIS PAPER...........................................1
    *   CURRENT STATE OF THE INDUSTRY........

In [10]:
with open(r"..\data\txt\Understanding Credit Cards Fraud.txt", "w") as f:
    f.write(''.join(text))